In [ ]:
%load_ext autoreload
%autoreload 2

# BaseCameraInterface

> A uniform, backend-agnostic contract for any camera driver in your photonics lab software. By subclassing the abstract base class, you can swap in different SDKs (FliSdk, Flir/spinaker, or even a test stub) without changing your acquisition pipeline.

In [ ]:
#| default_exp BaseCameraInterface

## Purpose

This module provides:

- A **uniform API** across all supported camera hardware.
- **SOLID** design principles to promote single-responsibility, extensibility, and decoupling.
- **Type safety** through Python type hints.
- **Easy testing** via dependency injection and mockability.

---

## Key Interface: `CameraInterface`

`CameraInterface` is an abstract base class defining the essential operations expected from any camera implementation.

### Core Lifecycle Methods

| Method                | Description                                                               |
|-----------------------|---------------------------------------------------------------------------|
| `__init__(**kwargs)` | Sets up camera SDK, applies configuration, and prepares for acquisition. |
| `start_capture()`     | Begins image acquisition or live streaming.                              |
| `stop_capture()`      | Halts image acquisition or live streaming.                               |
| `get_image(wait=True)`| Retrieves the latest frame, optionally blocking until a new one is ready.|
| `close()`             | Releases hardware resources and shuts down the camera safely.            |

---

### Camera Settings as Properties

Common camera parameters are exposed as Python properties, allowing both reading and writing through a uniform interface:

| Property     | Type   | Description                          |
|--------------|--------|--------------------------------------|
| `exposure`   | float  | Camera exposure time (in seconds).   |
| `framerate`  | float  | Frame rate (in frames per second).   |
| `gain`       | float  | Sensor analog gain.                  |

These properties standardize hardware parameter control across heterogeneous devices.

---

### Dark Frame Management

`CameraInterface` includes methods for managing dark-frame capture and reuse:

| Method                        | Description                                                                 |
|-------------------------------|-----------------------------------------------------------------------------|
| `take_dark(frames, save_path)`| Captures and averages multiple frames to generate a dark frame.             |
| `load_dark(file)`             | Loads a previously saved dark frame from disk for subtraction or QA.       |

This is particularly useful for calibration routines and background subtraction workflows.

---

## Design Considerations

- **Interface Enforcement**: All camera drivers must adhere to the same method/property interface, enabling interchangeability without modification to upstream systems.
- **Extensibility**: New camera types can be integrated by subclassing `CameraInterface` and implementing the required methods.
- **Mockability for Testing**: By abstracting away hardware-specific logic, the interface enables injection of mocks and stubs in test environments.
- **Consistency**: Centralized documentation and type annotations help enforce clarity and uniformity across all implementations.

---

## Intended Usage

The module is meant to be subclassed by hardware-specific drivers. This approach enables higher-level tools (e.g. data acquisition scripts, GUIs, or analysis pipelines) to remain agnostic to the underlying camera backend.

For a practical implementation, refer to concrete subclasses such as `FliCamera`, which adapts the interface to the FliSdk hardware.

---

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
#| hide

# bring forth **kwargs from an inherited class for documentation
from fastcore.meta import delegates
from fastcore.foundation import patch

from abc import ABC, abstractmethod
from pathlib import Path
from typing import Any, Optional, Tuple, Union
import numpy as np

In [ ]:
#| exporti BaseCameraInterface

@delegates()
class BaseCameraInterface(ABC):
    """
    Abstract camera interface defining standard operations
    for different camera backends,  with
    context-manager support for Python’s `with` statement.
    """

    def __init__(self,       
                 camera_index=0,
                 camera_id=None,
                 verbose=False,
                 cam_settings=None,
                 cropdims=None,
                 darkpath="./",
                 darkfile=None,
                 **kwargs):
        """
        Initialise the camera with given parameters.
        """
        ...
    
    def __enter__(self) -> "CameraInterface":
        """
        Enter the runtime context related to this object.
        Automatically starts capture upon entering.
        """
        return self
    
    def __exit__(
        self,
        exc_type: Optional[type],
        exc_value: Optional[BaseException],
        traceback: Optional[Any]
    ) -> None:
        """
        Exit the runtime context and clean up.
        Automatically stops capture and closes the camera.
        """
        try:
            self.stop_capture()
        finally:
            self.close()
    
        
    @abstractmethod
    def start_capture(self) -> None:
        """
        Begin image acquisition or live stream.
        """
        ...
    
    
    @abstractmethod
    def stop_capture(self) -> None:
        """
        Halt image acquisition or live stream.
        """
        ...
    
    @property
    @abstractmethod
    def exposure(self) -> float:
        """Get or set the camera exposure time in seconds."""
        ...
    
    @exposure.setter
    @abstractmethod
    def exposure(self, value: float) -> None:
        ...
    
    @property
    @abstractmethod
    def framerate(self) -> float:
        """Get or set the camera frame rate in frames per second."""
        ...
    
    
    @framerate.setter
    @abstractmethod
    def framerate(self, value: float) -> None:
        ...
    
    @property
    @abstractmethod
    def gain(self) -> float:
        """Get or set the camera gain (e.g., sensor analog gain)."""
        ...
    
    @gain.setter
    @abstractmethod
    def gain(self, value: float) -> None:
        ...
    
    @abstractmethod
    def get_image(
        self,
        wait: bool = True
    ) -> np.ndarray:
        """
        Retrieve the latest image frame.
        :param wait: block until a new frame is available
        :return: 2D numpy array representing the image
        """
        ...
    
    @abstractmethod
    def take_dark(self, frames: int = 100, save_path: Optional[Path] = None) -> np.ndarray:
        """
        Acquire a dark frame by averaging multiple captures.
        :param frames: number of frames to average
        :param save_path: optional file path to save the dark frame
        :return: dark frame array
        """
        ...
    
    @abstractmethod
    def load_dark(self, file: Union[str, Path]) -> None:
        """
        Load a previously saved dark frame from disk.
        """
        ...
    
    @abstractmethod
    def close(self) -> None:
        """
        Release all resources and shut down the camera.
        """
        ...

In [ ]:
show_doc(BaseCameraInterface.start_capture)

---

[source](https://github.com/SAIL-Labs/sail-cameras/blob/main/sail_cameras/BaseCameraInterface.py#L63){target="_blank" style="float:right; font-size:smaller"}

### BaseCameraInterface.start_capture

>      BaseCameraInterface.start_capture ()

*Begin image acquisition or live stream.*

In [ ]:
show_doc(BaseCameraInterface.stop_capture)

---

[source](https://github.com/SAIL-Labs/sail-cameras/blob/main/sail_cameras/BaseCameraInterface.py#L70){target="_blank" style="float:right; font-size:smaller"}

### BaseCameraInterface.stop_capture

>      BaseCameraInterface.stop_capture ()

*Halt image acquisition or live stream.*

In [ ]:
show_doc(BaseCameraInterface.exposure)

---

[source](https://github.com/SAIL-Labs/sail-cameras/blob/main/sail_cameras/BaseCameraInterface.py#L84){target="_blank" style="float:right; font-size:smaller"}

### BaseCameraInterface.exposure

>      BaseCameraInterface.exposure ()

*Get or set the camera exposure time in seconds.*

In [ ]:
show_doc(BaseCameraInterface.framerate)

---

[source](https://github.com/SAIL-Labs/sail-cameras/blob/main/sail_cameras/BaseCameraInterface.py#L95){target="_blank" style="float:right; font-size:smaller"}

### BaseCameraInterface.framerate

>      BaseCameraInterface.framerate ()

*Get or set the camera frame rate in frames per second.*

In [ ]:
show_doc(BaseCameraInterface.gain)

---

[source](https://github.com/SAIL-Labs/sail-cameras/blob/main/sail_cameras/BaseCameraInterface.py#L106){target="_blank" style="float:right; font-size:smaller"}

### BaseCameraInterface.gain

>      BaseCameraInterface.gain ()

*Get or set the camera gain (e.g., sensor analog gain).*

In [ ]:
show_doc(BaseCameraInterface.get_image)

---

[source](https://github.com/SAIL-Labs/sail-cameras/blob/main/sail_cameras/BaseCameraInterface.py#L110){target="_blank" style="float:right; font-size:smaller"}

### BaseCameraInterface.get_image

>      BaseCameraInterface.get_image (wait:bool=True)

*Retrieve the latest image frame.
:param wait: block until a new frame is available
:return: 2D numpy array representing the image*

In [ ]:
show_doc(BaseCameraInterface.take_dark)

---

[source](https://github.com/SAIL-Labs/sail-cameras/blob/main/sail_cameras/BaseCameraInterface.py#L122){target="_blank" style="float:right; font-size:smaller"}

### BaseCameraInterface.take_dark

>      BaseCameraInterface.take_dark (frames:int=100,
>                                     save_path:Optional[pathlib.Path]=None)

*Acquire a dark frame by averaging multiple captures.
:param frames: number of frames to average
:param save_path: optional file path to save the dark frame
:return: dark frame array*

In [ ]:
show_doc(BaseCameraInterface.load_dark)

---

[source](https://github.com/SAIL-Labs/sail-cameras/blob/main/sail_cameras/BaseCameraInterface.py#L132){target="_blank" style="float:right; font-size:smaller"}

### BaseCameraInterface.load_dark

>      BaseCameraInterface.load_dark (file:Union[str,pathlib.Path])

*Load a previously saved dark frame from disk.*

In [ ]:
show_doc(BaseCameraInterface.close)

---

[source](https://github.com/SAIL-Labs/sail-cameras/blob/main/sail_cameras/BaseCameraInterface.py#L139){target="_blank" style="float:right; font-size:smaller"}

### BaseCameraInterface.close

>      BaseCameraInterface.close ()

*Release all resources and shut down the camera.*

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()